# Quick baseline

In [1]:
import os
import sys
import torch
import monai
import ignite
import logging

from quick.utils import load_config
from quick.train import SegmentationTrainer
from quick.data import segmentation_dataloaders

In [2]:
config = load_config()
monai.utils.set_determinism(seed=config.seed)

In [3]:
trainer=SegmentationTrainer(
    progress_bar=True, 
    early_stopping = True, 
    metrics = ["MeanDice", "HausdorffDistance", "SurfaceDistance"],
    save_latest_metrics = True,
    config=config
)

In [4]:
trainer.run()

resuming from previous checkpoint at models/network_test_key_metric=0.0354.pt


[1/25]   4%|4          [00:00<?]

[1/5]  20%|##         [00:00<?]

[1/25]   4%|4          [00:00<?]

[1/5]  20%|##         [00:00<?]

...

[1/5]  20%|##         [00:00<?]

[1/25]   4%|4          [00:00<?]

[1/5]  20%|##         [00:00<?]

[1/25]   4%|4          [00:00<?]

[1/5]  20%|##         [00:00<?]

In [5]:
test_dl = segmentation_dataloaders(train=False, valid=False, test=True)

In [ ]:
trainer.evaluate(
    checkpoint='models/network_anatomy_2_key_metric=0.8173.pt',
    dataloader=test_dl
)

In [ ]:
outputs = {k : [o[0][k].detach().cpu().squeeze() for o in trainer.evaluator.state.output] for k in ['image', 'label', 'pred']}

In [ ]:
from quick.viewer import ListViewer

In [ ]:
ListViewer(
    [o.transpose(0,2).flip(-2) for o in outputs['image'][0:3]] + 
    [o.argmax(0).transpose(0,2).flip(-2).float() for o in outputs['label'][0:3]] + 
    [o.argmax(0).transpose(0,2).flip(-2).float() for o in outputs['pred'][0:3]] 
).show()